In [1]:
import sys
from typing import Self
from dataclasses import dataclass

sys.path.append('../utils')
from pyutils import *

In [2]:
sample = """RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE
"""

In [3]:
puzzle = readutf8('input.txt')

In [4]:
@dataclass
class GardenRegion:
    plant: str
    plots: list[Point]
    perimeter: int
    area: int

    def _find_perimeter(self) -> int:
        perimeter: int = 0
        for pt in self.plots:
            plot_sides: int = 4
            for pt2 in self.plots:
                if pt == pt2:
                    continue
                if (pt[0] == pt2[0]) and (abs(pt[1] - pt2[1]) == 1):
                    plot_sides -= 1
                if (pt[1] == pt2[1]) and (abs(pt[0] - pt2[0]) == 1):
                    plot_sides -= 1
            perimeter += plot_sides
        return perimeter

    @classmethod
    def from_list(cls, data: list[tuple[Point, str]]) -> Self:
        inst = GardenRegion(plant=data[0][1], plots=[pt for pt, _ in data], perimeter=0, area=0)
        inst.area = len(inst.plots)
        inst.perimeter = inst._find_perimeter()
        return inst

In [1]:
def find_regions(mat: Matrix) -> list[GardenRegion]:
    pts_checked: list[Point] = []
    regions: list[list[tuple[Point, str]]] = []

    print(f'LEN: {len(list(mat_iter(mat)))}')

    def _search_branches(pt: Point, mat: Matrix, plot: str) -> list[tuple[Point, str]]:
        if pt in pts_checked:
            return list()
        pts_checked.append(pt)
        region: list[tuple[Point, str]] = [(pt, plot)]
        adj_matching = {k:v for k, v in points_adjacent(pt, mat, corners=False).items() if (k not in pts_checked) and (v == plot)}
        for branch_pt, _ in adj_matching.items():
            region.extend(_search_branches(branch_pt, mat, plot))
        return region

    for pt, plot in mat_iter(mat):
        if pt in pts_checked:
            continue
        search_result = _search_branches(pt, mat, plot)
        if search_result:
            regions.append(search_result)

    return [GardenRegion.from_list(data) for data in regions]

NameError: name 'Matrix' is not defined

In [14]:
garden: Matrix = strtomat(puzzle)

In [15]:
timer = Stopwatch('region creation')
garden_regions = find_regions(garden)
timer.print_lap()

LEN: 19600
Stopwatch region creation: 34.84168495


In [16]:
print(sum(r.area * r.perimeter for r in garden_regions))

1449902
